# DenseNet Application of Image Classification

In [5]:
from torchvision.models.densenet import densenet121 as feature_extractor
from torch import nn


## Config

In [10]:
SCALE = 256
TRAIN = "/data/avatar/train/"
VALID = "/data/avatar/valid/"

In [2]:
conv_model = feature_extractor(pretrained=True)

/home/zhangxiaochen/anaconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


### Split ConvLayer to 2 parts, conv0~transtition3, (not gonna train), denseblock4~norm5 (train)

In [6]:
dense_conv1 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["conv0","norm0","relu0","pool0","denseblock1","transition1",
                                                                                   "denseblock2","transition2","denseblock3","transition3",]])

dense_conv2 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["denseblock4","norm5"]])

### Loading data

In [8]:
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [ ]:
transform = transforms.Compose([transforms.Resize(SCALE,SCALE),])

In [ ]:
trn = ImageFolder(TRAIN,transform=)